In [ ]:
#Requirements :
!pip install praw
!pip install xmltodict

Le sujet choisi est : Machine Learning

Partie 1  :

In [37]:
# Importer le module
import Document

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(Document)

<module 'Document' from '/content/Document.py'>

In [42]:
#Reddit
import praw
from datetime import datetime
import Author

#instanciation de l'authentification aux outils API de Reddit
reddit = praw.Reddit(client_id='WhOeUX8xCa_LSqqzHogeNA', client_secret='IouRe-5putFpKpyr11rOW7Wzh2Rpmw', user_agent='Web Scrapping')

#Les 10 posts les plus tendance du subreddit MachineLearning :
hot_posts = reddit.subreddit('MachineLearning').hot(limit=10)

#variables principales
docs = []                 #stocke les données d'un article (indice 0 → titre, 1 → auteur, 2 → date de publication, 3 → url, 4 → contenu textuel)
origines = []             #tableau indiquant l'origine du document (Reddit ou Arxiv)
id2doc = {}               #Clés : id du document, Valeurs : objet Document
id2aut = {}               #Clés : noms des auteurs, Valeurs : id des documents publiés
id = 1                    #id du document

#pour chaque post Reddit
for posts in hot_posts:
  docs.append(posts.title)                #ajout du titre dans docs
  auteurs = posts.author.name             #ajout du nom d'auteur dans docs
  docs.append(auteurs)

  texte = posts.selftext                  #contenu texte
  texte = texte.replace("\n", " ")                #formatage du texte pour remplacer les sauts de ligne \n par un espace
  texte = texte.replace("\t", " ")
  texte = texte.replace(";", " ")
  texte = texte.replace("&#x200B", "")

  if auteurs not in id2aut.keys():        #vérification de la présence de l'auteur dans id2aut
    id2aut[auteurs] = Author.Author(auteurs)  #instanciation de l'objet Author avec le nouvel auteur
    id2aut[auteurs].add(id, texte)
  else:
    id2aut[auteurs].add(id, texte)           #ajout du document à la production de l'auteur

  dateP = posts.created_utc               #ajout de la date dans docs au format unix
  docs.append(datetime.fromtimestamp(dateP))

  docs.append(posts.url)                  #ajout de l'url dans docs

  docs.append(texte)                      #ajout du contenu texte dans docs
  docs.append(posts.num_comments)         #ajout du nombre commentaire dans docs

  docs.append("Reddit")                   #récupération de l'origine du post

  #Une fois toutes les données du post récupérées, on instancie l'objet Document dans id2doc avec un id unique
  id2doc[id] = Document.RedditDocument(docs[0], docs[1], docs[2], docs[3], docs[4], docs[5], docs[6])
  id+=1                                   #incrémentation de l'id
  docs = []                               #on vide le tableau pour pouvoir ajouter les données du document suivant


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Champs disponibles pour Reddit API :
* 'title' (titre du post)
* 'score' (nombre d'upvotes)
* 'id' (identifiant du post, contenu dans l'url)
* 'subreddit' (communauté)
* 'url' (lien dans le post)
* 'num_comments' (nb commentaires)
* 'body' (contenu commentaire)
* 'selftext' (contenu du post si c'est textuel)
* 'author' (nom de l'utilisateur auteur du post)
* created_utc (heure de publication du post)

In [43]:
#Arxiv
import requests
import xmltodict
import dateutil.parser as dp

#url contenant la requete (ici, 10 articles résultant de la recherche des mots Machine et Learning).
url = 'http://export.arxiv.org/api/query?search_query=all:machine+learning&start=0&max_results=10'

data = requests.get(url)                      #Obtention des données au format XML à partir de la requête
articles = xmltodict.parse(data.text)         #données converties du format XML à dictionnaire

#parcours des articles
entries = articles['feed']['entry']
for entry in entries:
  a2 = [] #tableau des coauteurs
  docs.append(entry.get("title"))             #ajout du titre dans docs

  summary = entry['summary']                  #contenu textuel
  summary = summary.replace("\n", " ")
  summary = summary.replace("\t", " ")                  #formatage du texte pour remplacer les sauts de ligne \n par un espace
  summary = summary.replace(";", " ")
  summary = summary.replace("&#x200B", "")

  authors = entry.get("author", [])           #ajout de l'auteur dans docs
  if type(authors) is dict:                   #si un seul auteur
    a = authors.get("name") #a = premier auteur
    if a not in id2aut.keys():                #vérification de la présence de l'auteur dans id2aut
      id2aut[a] = Author.Author(a)  #instanciation de l'objet AUthor avec le nouvel auteur
      id2aut[a].add(id, summary)
    else:
      id2aut[a].add(id, summary)
    docs.append(a)
  else:                                       #si plusieurs auteurs
    docs.append(authors[0].get("name"))
    for au in authors:
      a2.append(au.get("name"))
      if au.get("name") not in id2aut.keys(): #vérification de la présence de l'auteur dans id2aut
        id2aut[au.get("name")] = Author.Author(a2)
        id2aut[au.get("name")].add(id, summary)
      else:
        id2aut[au.get("name")].add(id, summary)
    a2 = a2[1:]                              #on met la lste des co-auteurs dans a2 (donc tous sauf le premier element)

  dateP = entry.get("published")              #ajout de la date de publication dans docs au format unix
  dateP = dp.parse(dateP)
  dateP = dateP.timestamp()
  docs.append(datetime.fromtimestamp(dateP))

  url = entry.get("id")                       #ajout de l'url dans docs
  docs.append(url)

  docs.append(summary)                        #ajout du contenu textuel dans docs

  docs.append(a2)

  docs.append("Arxiv")                        #récupération de l'origine du post

  #Une fois toutes les données du post récupérées, on instancie l'objet Document dans id2doc avec un id unique
  id2doc[id] = Document.ArxivDocument(docs[0], docs[1], docs[2], docs[3], docs[4], docs[5], docs[6])
  docs = []                                   ##on vide le tableau pour pouvoir ajouter les données du document suivant
  id+=1                                       #incrémentation de l'id


Champs disponibles pour Arxiv :

* author/name (nom d'un auteur de l'article)
* title (titre de l'article)
* id (identifiant de l'article)
* published (date de première publication)
* link (lien de l'article)
* update (date de mise à jour de l'article)
* summary (sommaire du contenu de l'article)


Classe Document :

In [45]:
d1 = id2doc[13]     #test avec le 13ème document sur 20
print()
d1.__str__()   #affiche les attributs de l'objet
d1.getType()
print(d1.coAuteur)
print(d1.auteur)




Titre : Minimax deviation strategies for machine learning and recognition with
  short learning samples
Auteur : Michail Schlesinger
Date : 2017-07-16
Url : http://arxiv.org/abs/1707.04849v1
Texte : The article is devoted to the problem of small learning samples in machine learning. The flaws of ma...
Evgeniy Vodolazskiy
Michail Schlesinger


Classe Author :

In [46]:
print("Liste complète des auteurs :")                                #affiche la liste des noms d'auteurs
for key in id2aut:
  print("-", key)

verif = False
print()
while verif is False:                                                #boucle pour que l'utilisateur entre un nom d'auteur valide
  nom  = str(input("Entrez un nom d'auteur parmis ceux proposés : "))
  if nom in id2aut.keys():
    verif = True
  else:
    verif = False
    print("Veuillez entrer un nom d'auteur valide parmi la liste proposée.")
    print()

a1 = id2aut[nom]                                             #instanciation de l'auteur choisi
a1.get_taille_moyenne_documents()                            #affichage des informations concernées

Liste complète des auteurs :
- AutoModerator
- Rajivrocks
- SublimeSupernova
- AgeOfEmpires4AOE4
- Extension-Aspect9977
- ashz8888
- Pranav_999
- DataPastor
- Fair-Rain3366
- Elad Hazan
- Xiaojin Zhu
- Michail Schlesinger
- Evgeniy Vodolazskiy
- Wei-Hung Weng
- Samiyuru Menik
- Lakshmish Ramaswamy
- Amnon Shashua
- Ayaz Akram
- Jason Lowe-Power
- Randy J. Chase
- David R. Harrison
- Amanda Burke
- Gary M. Lackmann
- Amy McGovern
- Jindong Gu
- Daniela Oelke
- Dustin Juliano

Entrez un nom d'auteur parmis ceux proposés : AutoModerator


'Nombre de documents produits : 2 - Taille moyenne des documents : 594 caractères'

In [47]:
# Importer le module
import Corpus

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(Corpus)

<module 'Corpus' from '/content/Corpus.py'>

In [48]:
#Instanciation de corpus
corpus = Corpus.Corpus("Machine Learning", id2aut, id2doc)

corpus.save("data.csv")

corpus.afficher_par_date(19)
print()
corpus.afficher_par_titre(19)
print()



Tri par date :
- 2025-11-10 | [D] ML Pipelines completely in Notebooks within Databricks, thoughts? (Rajivrocks)
- 2025-11-10 | [D] Information geometry, anyone? (SublimeSupernova)
- 2025-11-10 | [P] SDLArch-RL is now compatible with Citra!!!! And we'll be training Street Fighter 6!!! (AgeOfEmpires4AOE4)
- 2025-11-10 | [D] AAAI-26 Student Scholar Volunteer Program (Extension-Aspect9977)
- 2025-11-10 | Unsure about submitting to TMLR[R] (Pranav_999)
- 2025-11-10 | [R] AlphaEvolve: Breaking 56 Years of Mathematical Stagnation (Fair-Rain3366)
- 2025-11-09 | [P] RLHF (SFT, RM, PPO) with GPT-2 in Notebooks (ashz8888)
- 2025-11-09 | [D] Which programming languages have you used to ship ML/AI projects in the last 3 years? (DataPastor)
- 2025-11-02 | [D] Self-Promotion Thread (AutoModerator)
- 2025-10-31 | [D] Monthly Who's Hiring and Who wants to be Hired? (AutoModerator)
- 2023-01-23 | Towards Modular Machine Learning Solution Development: Benefits and
  Trade-offs (Samiyuru Menik)
- 2022-04

Partie 4: Tester le singleton

In [49]:
corpus2 = Corpus.Corpus("Machine_Learning_2", id2aut, id2doc)
print(corpus is corpus2) # retourne True si la même instance est crée (verifie qu'il est singleton)

True


Générateur de documents grˆace `a un patron de conception d’usine (factory pattern)

In [50]:
# Importer le module
import DocumentFactory

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(DocumentFactory)

<module 'DocumentFactory' from '/content/DocumentFactory.py'>

Tester le generateur avec reddit et arxiv

In [52]:
from DocumentFactory import DocumentFactory  # importe la classe

id2doc[id] = DocumentFactory.createDocument("arxiv", titre="Le sujet du monde IA", auteur="Aissatou", date=datetime.today(),url="https://arxiv.com/xyz", texte="loremmmmmmmmmm, je fais le tour du monde Texte du post.lom..", extra=["Bob", "Charlie"])

id+=1                                       #incrémentation de l'id

id2doc[id] = DocumentFactory.createDocument("reddit", titre="Le tour du monde IA", auteur="Barry", date=datetime.today(),url="https://reddit.com/xyz", texte="je fais le tour du monde Texte du post.lom..", extra=50)

id+=1                                       #incrémentation de l'id


In [53]:
d1 = id2doc[22]     #test avec le 22ème document sur 24
print()
d1.__str__()   #affiche les attributs de l'objet
d1.getType()



Titre : Le tour du monde IA
Auteur : Barry
Date : 2025-11-10
Url : https://reddit.com/xyz
Texte : je fais le tour du monde Texte du post.lom.....


'Reddit'